
# **Econometría Aplicada Avanzada**

## Práctica calificada 1: Diferencias en Diferencias

### **Integrantes**

1. Gianfranco Romero (20196091)
2. Shaska Guevara (20171597)
3. Micaela Gutierrez (20171488)
4. Eduardo Ramírez (20162936)
5. Fernando Mendoza (20105246)

### **Librerías**

Instalamos las librerías

In [ ]:
!pip install csdid
!pip install pandas
!pip install statsmodels
!pip install linearmodels
!pip install matplotlib
!pip install pyfixest
!pip install drdid
!pip install requests
!pip install pyreadstat

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.3/748.3 kB 27.9 MB/s eta 0:00:00
  Created wheel for csdid: filename=csdid-0.2.5-py3-none-any.whl size=22349 sha256=5ab341b5711fc5226e316c81b19853eb31fc426ef1b4d9e476440c683273c3ea
  Stored in directory: /root/.cache/pip/wheels/b7/ed/ce/85e1d7548d898fddfa839d8947d1be93af80b8c62d3e724da3
Successfully built csdid
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: t

Y luego las importamos

In [1]:
from csdid.att_gt import ATTgt
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import warnings
warnings.filterwarnings("ignore")
import csdid
import pyfixest as pf
import requests
import pyreadstat
import statsmodels.formula.api as smf
import scipy.stats as stats
from scipy import stats
from statsmodels.sandbox.regression.gmm import IV2SLS
from statsmodels.sandbox.regression.gmm import IVGMM

Cargamos la base de datos

In [3]:
# Creamos un enlace directo al archivo usando el servicio raw de GitHub
url = "https://github.com/micaelagn/Micaela-G/raw/refs/heads/main/PC2/card.dta"

# Cargamos el archivo .dta
data = pd.read_stata(url)

# Y verificamos que se cargó correctamente mostrando las primeras filas
print(data.head())

   id  nearc2  nearc4  educ  age  fatheduc  motheduc    weight  momdad14  \
0   2       0       0     7   29       NaN       NaN  158413.0         1   
1   3       0       0    12   27       8.0       8.0  380166.0         1   
2   4       0       0    12   34      14.0      12.0  367470.0         1   
3   5       1       1    11   27      11.0      12.0  380166.0         1   
4   6       1       1    12   34       8.0       7.0  367470.0         1   

   sinmom14  ...  smsa66  wage  enroll   KWW     IQ  married  libcrd14  exper  \
0         0  ...       1   548       0  15.0    NaN      1.0       0.0     16   
1         0  ...       1   481       0  35.0   93.0      1.0       1.0      9   
2         0  ...       1   721       0  42.0  103.0      1.0       1.0     16   
3         0  ...       1   250       0  25.0   88.0      1.0       1.0     10   
4         0  ...       1   729       0  34.0  108.0      1.0       0.0     16   

      lwage  expersq  
0  6.306275      256  
1  6.17586

### 1. Realice una descripción de las características de los individuos de la muestra. <br> ¿Qué información nos provee el promedio de la dependiente en la interpretación de los coeficientes? <br>¿Cómo se interpretan los coeficientes en este caso?

In [5]:
# Estadísticas descriptivas generales
print(data.describe())  # Para variables numéricas

# Distribución de variables categóricas
categorical_cols = data.select_dtypes(include='category').columns
for col in categorical_cols:
    print(f"\nDistribución de {col}:")
    print(data[col].value_counts())

                id       nearc2       nearc4         educ          age  \
count  3010.000000  3010.000000  3010.000000  3010.000000  3010.000000   
mean   2581.748837     0.440864     0.682060    13.263455    28.119601   
std    1500.538849     0.496573     0.465753     2.676913     3.137004   
min       2.000000     0.000000     0.000000     1.000000    24.000000   
25%    1275.500000     0.000000     0.000000    12.000000    25.000000   
50%    2541.000000     0.000000     1.000000    13.000000    28.000000   
75%    3858.750000     1.000000     1.000000    16.000000    31.000000   
max    5225.000000     1.000000     1.000000    18.000000    34.000000   

          fatheduc     motheduc        weight     momdad14     sinmom14  ...  \
count  2320.000000  2657.000000  3.010000e+03  3010.000000  3010.000000  ...   
mean     10.003448    10.348137  3.211852e+05     0.789369     0.100664  ...   
std       3.720737     3.179671  1.706459e+05     0.407825     0.300934  ...   
min       0.0

In [7]:
print(f"Promedio de la dependiente (wage): {data['wage'].mean()}")


Promedio de la dependiente (wage): 577.2823920265781


### 2. Regresione usando OLS el siguiente modelo en Stata <br> `reg` `lwage` `educ` `c.exper##c.exper` `age` `black` `south` `smsa` , `smsa66`,`reg661-reg668`<br> ¿Qué ocurre con edad? ¿Por qué? <br> Ahora saque edad del modelo anterior y regresione de nuevo. <br> Interprete el estimado obtenido para educación, y compárelos con la `Tabla 2` y `Columna 2` del paper

In [9]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Crear el término cuadrático de exper
data['exper_sq'] = data['exper'] ** 2

# Modelo con edad
formula_1 = "lwage ~ educ + exper + exper_sq + age + black + south + smsa + smsa66 + reg661 + reg662 + reg663 + reg664 + reg665 + reg666 + reg667 + reg668"

# Ajustar el modelo
model_1 = smf.ols(formula=formula_1, data=data.dropna()).fit()

# Resumen del modelo
print(model_1.summary())


                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     27.87
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           4.00e-70
Time:                        19:13:50   Log-Likelihood:                -690.90
No. Observations:                1600   AIC:                             1414.
Df Residuals:                    1584   BIC:                             1500.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3522      0.009     41.103      0.0

In [11]:
# Modelo sin edad
formula_2 = "lwage ~ educ + exper + exper_sq + black + south + smsa + smsa66 + reg661 + reg662 + reg663 + reg664 + reg665 + reg666 + reg667 + reg668"

# Ajustar el modelo
model_2 = smf.ols(formula=formula_2, data=data.dropna()).fit()

# Resumen del modelo
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     27.87
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           4.00e-70
Time:                        19:13:54   Log-Likelihood:                -690.90
No. Observations:                1600   AIC:                             1414.
Df Residuals:                    1584   BIC:                             1500.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.8229      0.097     49.562      0.0

### 3. Vea la `Tabla 2` de Card (1993). ¿Qué se encuentra al inspeccionar el coeficiente estimado de educación? ¿Es creíble este coeficiente? ¿Por qué? <br> ¿Qué 2 posibles fuentes de inconsistencia podrían ocurrir como resultado de la omisión de variables? <br> Use el marco visto en clase (las expresiones que forman parte de $Inconsistencia=γ \frac{Cov(x,α)}{Var(x)} $) para responder y mencione la dirección esperada del sesgo.

Al inspeccionar el coeficiente esmitado de educación, se puede ver que los diferentes modelos propuestos muestran un efecto relativamente constante del efecto de cada año de educación sobre los ingresos. Así, cada regresión de OLS muestra que la educación incrementa los ingresos de los hombres encuestados entre 0.74 y 0.75.

En realidad, este coeficiente no es creíble. Esto debido a que la educación de cada hombre puede estar relacionada con otras variables que no son medidas pero que igualmente tienen un efecto sobre los ingresos. Es decir, no es creíble este coeficiente estimado de educación pues existe un problema de endogeneidad.

La omisión de variables puede dar pie a inconsistencia, que implica que el coeficiente estimado en la muestra realmente no nos dice nada sobre la población. Esta inconsistencia se dará si se cumplen dos condiciones: Primero, si existe una variable no observada que esté afectando la variable que se quiere predecir. Segundo, si alguna variable explicativa del modelo (en nuestro caso, educación) está relacionada con esa variable no observada.

Así, en nuestro caso, podemos detectar dos posibles fuentes de inconsistencia dadas por la omisión de variables.

a) La habilidad de una persona puede estar relacionada tanto con su educación como con sus ingresos. Si una persona acumula más años de estudios, es razonable pensar que esto le dará una mayor habilidad para realizar su trabajo. Además de ello, si una persona tiene más habilidades para el trabajo, entonces podrá ser más productivo y conseguir mayores ingresos. Así, siguiendo $\gamma $ positivo y $\theta_{1}$ igualmente positivo, la dirección del sesgo es positiva.

b) Las redes de contactos establecidas en los centros de estudios también están relacionadas con la educación y los ingresos. Lógicmanete, el pasar por un centro educativo, sobre todo uno donde los estudiantes de clase alta, abre la posibilidad de generar vínculos con estos estudiantes y a sus redes de contactos, los cuales son muy útiles al momento de conseguir empleos con mayor remuneración. A medida que una persona tienen más años de educación, esta tiene más tiempo para forjar más vínculos de este tipo. Además, al tener más vínculos, se abren mayores oportunidades para conseguir trabajos mejor pagados. Así, siguiendo $\gamma $ positivo y $\theta_{1}$ igualmente positivo, la dirección del sesgo es positiva.

### 4. Estime la ecuación de primera etapa <br> ¿Qué debemos ver en esta regresión? <br> ¿Qué podría decir respecto de la correlación parcial existente entre `educ` y `nearc4`?

In [13]:
# Ecuación de primera etapa sin controles

import statsmodels.api as sm

y_ols = data['educ']
X_ols = sm.add_constant(data['nearc4'])
model_first_stage = sm.OLS(y_ols, X_ols).fit()
print(model_first_stage.summary())

                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     63.91
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           1.84e-15
Time:                        19:14:01   Log-Likelihood:                -7202.7
No. Observations:                3010   AIC:                         1.441e+04
Df Residuals:                    3008   BIC:                         1.442e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.6980      0.086    148.269      0.0

Podemos ver que existe un coeficiente de 0.82 para nearc4. Existe una correlación positiva muy fuerte entre esta variable y los años de educación. Esto nos sugeriría que se trata de un buen instrumento para aplicar una estrategia de variables instrumentales. A continuación, incluiremos variables de control en el modelo para tener una mejor estimación.

In [15]:
y_ols = data['educ']
# Los controles son tomados de la Tabla 2 de Card (1993)
X_ols = sm.add_constant(data[['nearc4','momdad14','south','black','smsa','reg661','reg662','reg663','reg664','reg665','reg666','reg667','reg668','smsa66']])
model_first_stage = sm.OLS(y_ols, X_ols).fit()
print(model_first_stage.summary())


coef_nearc4 = model_first_stage.params['nearc4']
cov_nearc4 = model_first_stage.cov_params().loc['nearc4', 'nearc4']

f_stat = (coef_nearc4 ** 2) / cov_nearc4

df_num = 1
df_den = model_first_stage.df_resid

p_value = 1 - stats.f.cdf(f_stat, df_num, df_den)
print(f"F test of excluded instruments:\n F-statistic = {f_stat:.2f}\n Prob > F = {p_value:.4f}")

print("Stock-Yogo weak ID test critical values for single endogenous regressor:")
print("                                   10% maximal IV size             16.38")
print("                                   15% maximal IV size              8.96")
print("                                   20% maximal IV size              6.66")
print("                                   25% maximal IV size              5.53")
print("Source: Stock-Yogo (2005).")

                            OLS Regression Results                            
Dep. Variable:                   educ   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     32.88
Date:                Sun, 08 Dec 2024   Prob (F-statistic):           9.50e-83
Time:                        19:14:04   Log-Likelihood:                -7019.2
No. Observations:                3010   AIC:                         1.407e+04
Df Residuals:                    2995   BIC:                         1.416e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.4998      0.211     59.229      0.0

Al estimar la regresión con variables de control, podemos ver que el coeficiente de nearc4 se reduce a 0.4056. Es decir, tiene una correlación más débil con respecto a la educación. Es más, podemos ver que otras variables como haber vivido con ambos padres a los 14 años (*momdad14*), ser afroamericano (*black*) o vivir en una zona metropolitana (*smsa*) tienen coeficientes más fuertes que nearc4. Ahora bien, al ver el F-Statistic, vemos que es de 12.86. Al compararlo con los valores críticos de Stock y Yogo, vemos que la estimación de IV puede tener, como máximo, un 15% de sesgo del $\beta$ original.

### 5. Estime el modelo usando el estimador de Variables Instrumentales, usando `nearc4` como un instrumento para la variable educ. <br> Dibuje el DAG correspondiente y explique por qué se espera que esta estrategia funcione. <br> Comente sus resultados y compárelos con los obtenidos en el punto 2

Se observan diferencias importantes en la estimación del coeficiente de la variable educ (educación), lo que refleja el impacto de la endogeneidad en los resultados obtenidos mediante la regresión OLS. En el modelo OLS, el coeficiente estimado para la variable educación es de 19.62, lo que implica que, en promedio, un incremento en el nivel educativo está asociado con un aumento de 19.62 unidades en el salario. Sin embargo, esta estimación puede estar sesgada debido a la presencia de endogeneidad, que se refiere a la correlación entre el regressor (educación) y el error en el modelo. La endogeneidad puede surgir de factores no observados que afectan tanto al salario como a la educación, como habilidades innatas o factores no medidos que determinan tanto el nivel educativo como los ingresos, lo que lleva a una subestimación del verdadero impacto de la educación en el salario.

Por otro lado, el modelo IV, al utilizar el instrumento `nearc4` para corregir esta endogeneidad, proporciona una estimación significativamente mayor para el coeficiente de educación, alcanzando un valor de 61.39. Esto sugiere que, una vez corregido el sesgo de endogeneidad, el verdadero efecto de la educación sobre el salario es mucho mayor de lo que se indicaba en el modelo OLS. Es decir, el uso de un instrumento exógeno permite aislar el efecto causal de la educación sobre el salario.

En cuanto al R-cuadrado, se observa una marcada discrepancia entre los dos modelos. El modelo OLS tiene un R-cuadrado de 0.184, lo que indica que el 18.4% de la variabilidad en los salarios es explicada por las variables independientes del modelo, lo que podría sugerir un ajuste relativamente bueno, especialmente cuando no se tiene en cuenta la endogeneidad. En contraste, el modelo IV tiene un R-cuadrado de 0.026, lo que refleja una capacidad explicativa mucho menor del modelo. Este es un hallazgo típico en modelos IV, ya que el objetivo principal no es maximizar el ajuste, sino corregir la endogeneidad y obtener estimaciones causales más precisas.

Finalmente, tanto el modelo OLS como el modelo IV muestran que el coeficiente de educación es estadísticamente significativo, con p-valores muy bajos (prueba de hipótesis nula de que el coeficiente es igual a cero). Sin embargo, la confiabilidad de las estimaciones es considerablemente mayor en el modelo IV, dado que ha corregido el sesgo de endogeneidad. En resumen, aunque el modelo OLS puede ofrecer una buena predicción general del salario, el modelo IV proporciona una estimación más robusta y precisa del efecto causal de la educación, ajustada por la endogeneidad, lo que lo convierte en la opción preferida cuando se busca inferir relaciones causales más confiables.

In [67]:
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
from scipy import stats

# Primera etapa: Regresión de 'educ' sobre 'nearc4' y controles
# Variables dependientes y explicativas
y_first_stage = data['educ']
X_first_stage = sm.add_constant(data[['nearc4', 'momdad14', 'south', 'black', 'smsa', 
                                      'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 
                                      'reg666', 'reg667', 'reg668', 'smsa66']])

model_first_stage = sm.OLS(y_first_stage, X_first_stage).fit()

# Evaluación de la fuerza del instrumento (F-statistic)
coef_nearc4 = model_first_stage.params['nearc4']
cov_nearc4 = model_first_stage.cov_params().loc['nearc4', 'nearc4']
f_stat = (coef_nearc4 ** 2) / cov_nearc4
df_num = 1
df_den = model_first_stage.df_resid
p_value = 1 - stats.f.cdf(f_stat, df_num, df_den)

print(f"\nF test de exclusión del instrumento:\nF-statistic = {f_stat:.2f}\nProb > F = {p_value:.4f}")
print("Stock-Yogo weak ID test critical values para un regresor endógeno:")
print("                                   10% tamaño IV máximo             16.38")
print("                                   15% tamaño IV máximo              8.96")
print("                                   20% tamaño IV máximo              6.66")
print("                                   25% tamaño IV máximo              5.53")

# Segunda etapa: Regresión de 'wage' sobre los predictores usando IV
y_second_stage = data['wage']  # Variable dependiente
X_second_stage = data[['educ', 'momdad14', 'south', 'black', 'smsa', 
                       'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 
                       'reg666', 'reg667', 'reg668', 'smsa66']]  # Educ y controles
Z = data[['nearc4', 'momdad14', 'south', 'black', 'smsa', 
          'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 
          'reg666', 'reg667', 'reg668', 'smsa66']]  # Instrumentos y controles

iv_model = IV2SLS(y_second_stage, sm.add_constant(X_second_stage), sm.add_constant(Z)).fit()

print("\nResultados del estimador IV usando 'nearc4' como instrumento:")
print(iv_model.summary())

# Comparación con OLS (Pregunta 2)
ols_model = sm.OLS(y_second_stage, sm.add_constant(X_second_stage)).fit()
print("\nResultados del modelo OLS (de la pregunta 2):")
print(ols_model.summary())


F test de exclusión del instrumento:
F-statistic = 12.86
Prob > F = 0.0003
Stock-Yogo weak ID test critical values para un regresor endógeno:
                                   10% tamaño IV máximo             16.38
                                   15% tamaño IV máximo              8.96
                                   20% tamaño IV máximo              6.66
                                   25% tamaño IV máximo              5.53

Resultados del estimador IV usando 'nearc4' como instrumento:
                          IV2SLS Regression Results                           
Dep. Variable:                   wage   R-squared:                       0.026
Model:                         IV2SLS   Adj. R-squared:                  0.022
Method:                     Two Stage   F-statistic:                     33.04
                        Least Squares   Prob (F-statistic):           3.79e-83
Date:                Sun, 08 Dec 2024                                         
Time:                   

    nearc4
       ↓
     educ  ← Confusoras (U)
       ↓
     lwage


### 6. Testee la presencia de instrumentos débiles usando: <br> **i)** el estadístico F; <br> **ii)** el estadístico de Cragg y Donald (1993); <br> **iii)** las tablas de Stock y Yogo (2005) con respecto a la medida del test de Wald

#### i) el estadístico F

El estadístico F de la primera etapa es útil para identificar si el instrumento es relevante. Si el valor F es mayor a 10, no hay evidencia de que los instrumentos sean débiles. Dado que Prob > F = 0.0003 es mucho menor que 0.05, esto indica que el instrumento ``nearc4`` es relevante y no es débil. En otras palabras, el instrumento tiene una relación significativa con la variable explicativa ``educ`` y puede ser utilizado de manera confiable en el modelo de variables instrumentales (IV) 

In [78]:
# Primera etapa: Regresión de 'educ' sobre 'nearc4' y controles
y_first_stage = data['educ']
X_first_stage = sm.add_constant(data[['nearc4', 'momdad14', 'south', 'black', 'smsa', 
                                      'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 
                                      'reg666', 'reg667', 'reg668', 'smsa66']])

model_first_stage = sm.OLS(y_first_stage, X_first_stage).fit()

# Evaluación de la fuerza del instrumento (F-statistic)
coef_nearc4 = model_first_stage.params['nearc4']
cov_nearc4 = model_first_stage.cov_params().loc['nearc4', 'nearc4']
f_stat = (coef_nearc4 ** 2) / cov_nearc4
df_num = 1  # Numerador: 1 grado de libertad por instrumento
df_den = model_first_stage.df_resid  # Denominador: grados de libertad residuales
p_value = 1 - stats.f.cdf(f_stat, df_num, df_den)

print(f"\nF test de exclusión del instrumento:\nF-statistic = {f_stat:.2f}\nProb > F = {p_value:.4f}")


F test de exclusión del instrumento:
F-statistic = 12.86
Prob > F = 0.0003


#### ii) el estadístico de Cragg y Donald (1993)

Dado que el valor p es 0.0000, podemos rechazar la hipótesis nula de que los instrumentos son débiles. Esto significa que los instrumentos que estás utilizando son fuertes y no presentan problemas de identificación débil en el modelo.

In [84]:
import numpy as np
import statsmodels.api as sm

# Primera etapa: Regresión de 'educ' sobre 'nearc4' y controles
y_first_stage = data['educ']
X_first_stage = sm.add_constant(data[['nearc4', 'momdad14', 'south', 'black', 'smsa', 
                                      'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 
                                      'reg666', 'reg667', 'reg668', 'smsa66']])

# Regresión OLS de la primera etapa
model_first_stage = sm.OLS(y_first_stage, X_first_stage).fit()

# Obtener los residuos de la primera etapa
residuals_first_stage = model_first_stage.resid

# Estimación de la matriz de varianzas-covarianzas de los instrumentos
instrument_matrix = sm.add_constant(data[['nearc4', 'momdad14', 'south', 'black', 'smsa', 
                                          'reg661', 'reg662', 'reg663', 'reg664', 'reg665', 
                                          'reg666', 'reg667', 'reg668', 'smsa66']])
cov_matrix = np.linalg.inv(instrument_matrix.T @ instrument_matrix)

# Estadístico de Cragg y Donald
n = len(data)  # Número de observaciones
k = len(instrument_matrix.columns)  # Número de instrumentos
residual_sum_of_squares = residuals_first_stage.T @ residuals_first_stage  # Suma de cuadrados de los residuos

# Estadístico de Cragg y Donald (1993)
cragg_donald_statistic = (residual_sum_of_squares / n) * (n - k)  # Estadístico de Cragg y Donald

# Valor crítico (aproximado) para la distribución chi-cuadrada
dof = k - 1  # Grados de libertad (número de instrumentos - 1)
p_value_cragg_donald = 1 - stats.chi2.cdf(cragg_donald_statistic, df=dof)

print(f"\nEstadístico de Cragg y Donald (1993): {cragg_donald_statistic:.2f}")
print(f"P-value para el estadístico de Cragg y Donald: {p_value_cragg_donald:.4f}")



Estadístico de Cragg y Donald (1993): 18596.42
P-value para el estadístico de Cragg y Donald: 0.0000


#### iii) las tablas de Stock y Yogo (2005) con respecto a la medida del test de Wald

In [90]:
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

# Ajustar el modelo IV
iv_model = IV2SLS(y_second_stage, sm.add_constant(X_second_stage), sm.add_constant(Z)).fit()

# Supongamos que quieres probar si el coeficiente de 'nearc4' es cero
# r_matrix es una matriz que tiene una fila por cada restricción y una columna por cada parámetro del modelo
# Para probar si el coeficiente de 'nearc4' es cero en el modelo IV

# Usamos get_loc para obtener la posición de 'nearc4' en el índice
r_matrix = np.zeros((1, len(iv_model.params)))
r_matrix[0, iv_model.params.get_loc('nearc4')] = 1  # Esto especifica que probamos el coeficiente de 'nearc4'

# Prueba de Wald utilizando el método directamente del modelo
wald_stat = iv_model.wald_test(r_matrix)

# Valor de la estadística y p-valor
wald_value = wald_stat.statistic[0]
wald_pvalue = wald_stat.pvalue

print(f"\nEstadístico de Wald: {wald_value:.2f}")
print(f"P-value de la prueba de Wald: {wald_pvalue:.4f}")

# Valores críticos de Stock y Yogo (2005) para 1 instrumento endógeno
print("\nValores críticos de Stock y Yogo (2005):")
print("Para el nivel de significancia de 10%: Valor crítico = 16.38")
print("Para el nivel de significancia de 15%: Valor crítico = 8.96")
print("Para el nivel de significancia de 20%: Valor crítico = 6.66")
print("Para el nivel de significancia de 25%: Valor crítico = 5.53")

AttributeError: 'Series' object has no attribute 'get_loc'

### 7. **Use `nearc2` con `nearc4` como instrumentos para educ. <br> Primero estime la primera etapa para `educ` y analice cuál de los dos instrumentos está más fuertemente relacionada parcialmente con `educ`. <br> Después use el estimador de IV usando por separado `nearc2` y `nearc4` como instrumento para educ. <br> Luego use el de 2SLS (incluyendo ambos instrumentos). <br> Discuta sus resultados. Verifique que el estimador 2SLS es válido.**

In [22]:
# Primera etapa: Regresión de educ sobre los instrumentos (nearc2 y nearc4) y covariables relevantes
# Incluimos un intercepto automáticamente con statsmodels
model_educ_nearc2 = smf.ols('educ ~ nearc2', data=data).fit()
model_educ_nearc4 = smf.ols('educ ~ nearc4', data=data).fit()

# Resumen de las regresiones
summary_nearc2 = model_educ_nearc2.summary()
summary_nearc4 = model_educ_nearc4.summary()

summary_nearc2, summary_nearc4


(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                   educ   R-squared:                       0.002
 Model:                            OLS   Adj. R-squared:                  0.002
 Method:                 Least Squares   F-statistic:                     6.759
 Date:                Sun, 08 Dec 2024   Prob (F-statistic):            0.00937
 Time:                        19:20:09   Log-Likelihood:                -7231.0
 No. Observations:                3010   AIC:                         1.447e+04
 Df Residuals:                    3008   BIC:                         1.448e+04
 Df Model:                           1                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

En la primera etapa, ambos instrumentos (nearc2 y nearc4) muestran estar relacionados parcialmente con la educación (educ), pero con diferente intensidad. Para nearc2, el coeficiente estimado es 0.2553, lo que indica que vivir cerca de una universidad (según este indicador) incrementa los años de educación en promedio en 0.2553 años. Esta relación es estadísticamente significativa (𝑝 = 0.009), aunque la proporción de varianza explicada ($R^2$= 0.002) es baja, sugiriendo que su capacidad explicativa es limitada. Por otro lado, el coeficiente de nearc4 es 0.8290, mucho mayor que el de nearc2, indicando un efecto más fuerte sobre la educación. Además, este resultado es altamente significativo (𝑝 < 0.001) y presenta un $R^2$ = 0.021, lo que muestra una mayor capacidad explicativa que nearc2. Por lo tanto, nearc4 está más fuertemente relacionado con educ, y se perfila como un instrumento más robusto.

In [24]:
# Variables dependientes y explicativas
y = data['wage']  # Variable dependiente
X = data[['educ']]  # Variable explicativa

# IV con nearc2
Z_nearc2 = data[['nearc2']]  # Instrumento
iv_nearc2 = IV2SLS(y, sm.add_constant(X), sm.add_constant(Z_nearc2)).fit()

# IV con nearc4
Z_nearc4 = data[['nearc4']]  # Instrumento
iv_nearc4 = IV2SLS(y, sm.add_constant(X), sm.add_constant(Z_nearc4)).fit()

print("IV usando nearc2 como instrumento:")
print(iv_nearc2.summary())
print("\nIV usando nearc4 como instrumento:")
print(iv_nearc4.summary())

IV usando nearc2 como instrumento:
                          IV2SLS Regression Results                           
Dep. Variable:                   wage   R-squared:                      -4.494
Model:                         IV2SLS   Adj. R-squared:                 -4.496
Method:                     Two Stage   F-statistic:                     7.328
                        Least Squares   Prob (F-statistic):            0.00683
Date:                Sun, 08 Dec 2024                                         
Time:                        19:20:13                                         
No. Observations:                3010                                         
Df Residuals:                    3008                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2606.


En la estimación usando nearc2 como instrumento, el coeficiente de educ fue de 240.01, indicando que un año adicional de educación incrementa el salario promedio en 240 unidades monetarias. Este coeficiente es estadísticamente significativo (𝑝 = 0.007). Sin embargo, el $R^2$ = −4.494 sugiere que el modelo tiene problemas para explicar la variación en los salarios, lo que puede reflejar una debilidad del instrumento o un ajuste limitado del modelo. Por otro lado, al usar nearc4 como instrumento, el coeficiente de educ fue de 107.57, también positivo y significativo (𝑝 < 0.001), pero con un menor impacto relativo en comparación con nearc2. Además, el $R^2$ = −0.538 indica un ajuste del modelo algo mejor que el de nearc2, pero aún con problemas.

Al comparar los resultados, nearc4 parece ser un instrumento más fuerte y consistente, ya que produce un coeficiente significativo y con menor error estándar. En ambos casos, los coeficientes de educ respaldan la hipótesis de que la educación tiene un impacto positivo en los salarios, pero la calidad del ajuste del modelo es limitada, probablemente por las características de los instrumentos o por factores no capturados en el análisis.

In [26]:
# 2SLS con nearc2 y nearc4
Z_both = data[['nearc2', 'nearc4']]  # Ambos instrumentos
iv_both = IV2SLS(y, sm.add_constant(X), sm.add_constant(Z_both)).fit()

print("2SLS usando ambos instrumentos:")
print(iv_both.summary())

2SLS usando ambos instrumentos:
                          IV2SLS Regression Results                           
Dep. Variable:                   wage   R-squared:                      -0.689
Model:                         IV2SLS   Adj. R-squared:                 -0.689
Method:                     Two Stage   F-statistic:                     54.35
                        Least Squares   Prob (F-statistic):           2.16e-13
Date:                Sun, 08 Dec 2024                                         
Time:                        19:20:16                                         
No. Observations:                3010                                         
Df Residuals:                    3008                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -966.638

Al usar ambos instrumentos (nearc2 y nearc4) en un modelo 2SLS, el coeficiente de educ es 116.40, indicando que un año adicional de educación incrementa los salarios promedio en 116.4 unidades monetarias. Este coeficiente es altamente significativo (𝑝 < 0.001), lo que refuerza la relación positiva entre educación y salario. La fortaleza conjunta de los instrumentos está respaldada por un 𝐹-statistic de 54.35 (𝑝 < 0.001), lo que sugiere que el modelo cumple con la condición de relevancia instrumental. Sin embargo, el $R^2$ = − 0.689 indica que el modelo aún tiene dificultades para capturar la variación en los salarios, posiblemente debido a factores no modelados o limitaciones en la especificación. Este resultado muestra que combinar ambos instrumentos mejora la precisión de las estimaciones frente a usarlos de forma individual.

In [50]:
# Verificación de instrumentos
from statsmodels.stats.diagnostic import het_breuschpagan

# Residuales del modelo
resid_2sls = iv_both.resid
# Prueba de Breusch-Pagan para heterocedasticidad
bp_test = het_breuschpagan(resid_2sls, sm.add_constant(data[['educ', 'nearc2', 'nearc4']]))
labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))

{'Lagrange multiplier statistic': 9.442191564833468, 'p-value': 0.023954364427434042, 'f-value': 3.1531057063344305, 'f p-value': 0.023912439438320814}


Para evaluar la validez del estimador 2SLS, el resultado del test de sobreidentificación (Lagrange Multiplier Statistic o Sargan Test) tiene un valor estadístico de 9.44 y un 𝑝-valor de 0.0239, lo que indica que rechazamos la hipótesis nula de que los instrumentos son válidos al nivel de significancia del 5%. Esto sugiere que al menos uno de los instrumentos (nearc2 o nearc4) podría no ser válido, lo cual afecta la consistencia del estimador 2SLS. Además, el valor de 𝑓-statistic en la primera etapa es 3.15, con un 𝑝-valor de 0.0239, lo que señala que los instrumentos tienen relevancia débil en la predicción de educ. En conjunto, aunque los coeficientes obtenidos en el modelo 2SLS con ambos instrumentos son significativos y consistentes con las estimaciones individuales, estos resultados ponen en duda la fiabilidad de las estimaciones obtenidas con 2SLS. Esto sugiere la necesidad de reevaluar los instrumentos o considerar métodos más robustos para validar los resultados y garantizar conclusiones consistentes.

### 8. **Hemos asumido hasta ahora homoscedasticidad ¿Es razonable esto?¿Qué permite el estimador de GMM que no permite el de IV en 2 etapas bajo heterosk. Compare los resultados (del coeficiente educación) del estimador 2SLS con el del estimador GMM bajo heterocedasticidad**


In [52]:
# Estimador GMM con ambos instrumentos
gmm_model = IVGMM(y, sm.add_constant(X), sm.add_constant(Z_both))
gmm_result = gmm_model.fit()
print("GMM bajo heterocedasticidad:")
print(gmm_result.summary())

Optimization terminated successfully.
         Current function value: 273.616758
         Iterations: 14
         Function evaluations: 18
         Gradient evaluations: 18
Optimization terminated successfully.
         Current function value: 0.002301
         Iterations: 3
         Function evaluations: 7
         Gradient evaluations: 7
Optimization terminated successfully.
         Current function value: 0.002302
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.002302
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
GMM bajo heterocedasticidad:
                                IVGMM Results                                 
Dep. Variable:                   wage   Hansen J:                        6.930
Model:                          IVGMM   Prob (Hansen J):               0.00848
Method:                           GMM                

In [54]:
print("Coeficiente 2SLS (educ):", iv_both.params['educ'])
print("Coeficiente GMM (educ):", gmm_result.params[1])

Coeficiente 2SLS (educ): 116.40415798679044
Coeficiente GMM (educ): 116.33894403448211


El supuesto de homoscedasticidad puede no ser razonable en este caso, ya que las condiciones reales de los datos a menudo presentan heterocedasticidad, lo que podría afectar la consistencia de los estimadores clásicos como el 2SLS. El estimador GMM, a diferencia del 2SLS, es robusto frente a heterocedasticidad, ya que utiliza matrices de varianzas y covarianzas que no dependen de este supuesto. Al comparar los resultados, el coeficiente de educación bajo 2SLS es de 116.404, mientras que bajo GMM es de 116.339; ambos son muy similares, lo que sugiere que la heterocedasticidad no tiene un impacto significativo en la estimación del coeficiente en este caso. Sin embargo, el Hansen J-test del modelo GMM (valor: 6.93, p-valor: 0.0085) cuestiona la validez de los instrumentos, indicando que podrían no ser completamente exógenos, lo cual requiere un análisis más profundo para confirmar la robustez de los resultados.

### 9. Sigamos con el caso donde se tienen dos instrumentos. Dado lo hallado en términos de si los instrumentos por separado son débiles o no en la pregunta 7, ¿qué concluye ud. a partir del resultado del test de sobreidentificación? <br> ¿Con qué instrumento o instrumentos se quedaría al final? Realícelo asumiendo heterocedascidad para el estimador GMM y 2SLS <br> *(HINT: piense 2 veces)*

### 10. ¿Deberíamos usar MCO y aceptar la inconsistencia o usar en vez IV a pesar de que es un estimador consistente pero ineficiente? <br> ¿Hay forma de testear esto? <br> Si sí, llévela a cabo para el modelo sobre-identificado. ¿Qué concluye?